In [1]:
! pip install duckdb
! pip install pandas
! pip install seaborn
! pip install matplotlib
! pip install jupysql pandas matplotlib duckdb-engine

zsh:1: /Users/coen/Documents/dev/dashboard-analysis/.venv/bin/pip: bad interpreter: /Users/coen/Documents/dev/.venv/bin/python3.13: no such file or directory
zsh:1: /Users/coen/Documents/dev/dashboard-analysis/.venv/bin/pip: bad interpreter: /Users/coen/Documents/dev/.venv/bin/python3.13: no such file or directory
zsh:1: /Users/coen/Documents/dev/dashboard-analysis/.venv/bin/pip: bad interpreter: /Users/coen/Documents/dev/.venv/bin/python3.13: no such file or directory
zsh:1: /Users/coen/Documents/dev/dashboard-analysis/.venv/bin/pip: bad interpreter: /Users/coen/Documents/dev/.venv/bin/python3.13: no such file or directory
zsh:1: /Users/coen/Documents/dev/dashboard-analysis/.venv/bin/pip: bad interpreter: /Users/coen/Documents/dev/.venv/bin/python3.13: no such file or directory


In [2]:
import duckdb

%load_ext sql
conn = duckdb.connect()
%sql conn --alias duckdb
%sql INSTALL spatial; LOAD spatial;
%sql INSTALL excel; LOAD excel;
%config SqlMagic.displaylimit = None

Running query in 'duckdb'

Running query in 'duckdb'

displaylimit: Value None will be treated as 0 (no limit)

In [3]:
%%sql
DROP TABLE IF EXISTS YSL_data;
CREATE TABLE IF NOT EXISTS YSL_data AS
SELECT *
FROM st_read('../YSL data- Reinier Feitz.xlsx', layer='2023 all')


Running query in 'duckdb'

Count
8137


In [4]:
df = conn.sql("""
WITH YSL_data_OK_date AS (
    SELECT 
        * EXCLUDE ("OK dat", "Op OK", "Uit OK"),
        CAST(COALESCE(try_strptime("OK dat", '%Y/%d/%m'), strptime("OK dat", '%d-%m-%Y')) AS Date) AS date,
        "OK dat" AS old_date,
         epoch_ms(epoch_ms("Op OK") + epoch_ms(date)) AS actual_start_case_datetime,
         epoch_ms(epoch_ms("Uit OK") + epoch_ms(date)) AS actual_end_case_datetime
    FROM YSL_data
),
YLS_data_P_replace AS (
    SELECT * 
        REPLACE (upper(split_part("Verr. code", '.', 1)) AS "Verr. code")
    FROM YSL_data_OK_date
),
YSL_data_theatre_consistent AS (
    SELECT * EXCLUDE(OK), IF(contains(OK, '.'), split_part(OK, '.', 1), OK) AS theatre
    FROM YLS_data_P_replace
)
SELECT *, date FROM YSL_data_theatre_consistent
              WHERE P = 'E'
ORDER BY theatre, actual_start_case_datetime
""").to_df()

In [5]:
%%sql
DROP TABLE IF EXISTS YSL_data_protocol;
CREATE TABLE IF NOT EXISTS YSL_data_protocol AS
WITH YSL_data AS (
    SELECT * FROM read_csv('../P_IJsselland_Codes.csv')
),
YSL_data_codes_unnest AS (
    SELECT codes.split(',').unnest() AS protocol_code, 
        * EXCLUDE codes
    FROM YSL_data
),
YSL_data_distinct_protocols AS (
    SELECT DISTINCT 
        OrganizationName AS organization_name 
        ,organization_id
        ,procedure_id
        ,procedure_name
        , protocol_id
        -- ,protocol_name
        ,protocol_code
    FROM YSL_data_codes_unnest
)
SELECT *
FROM YSL_data_distinct_protocols
QUALIFY row_number() over (partition by protocol_code order by protocol_id) = 1

Running query in 'duckdb'

Count
1398


In [6]:
%%sql 
DROP TABLE IF EXISTS YSL_parsed_data;
CREATE TABLE YSL_parsed_data AS
WITH YSL_data_OK_date AS (
    SELECT 
        * EXCLUDE ("OK dat", "Op OK", "Uit OK"),
        CAST(COALESCE(try_strptime("OK dat", '%Y/%d/%m'), strptime("OK dat", '%d-%m-%Y')) AS Date) AS date,
         epoch_ms(epoch_ms("Op OK") + epoch_ms(date)) AS actual_start_case_datetime,
         epoch_ms(epoch_ms("Uit OK") + epoch_ms(date)) AS actual_end_case_datetime
    FROM YSL_data
),
YLS_data_P_replace AS (
    SELECT * 
        REPLACE(upper(split_part("Verr. code", '.', 1)) AS "Verr. code")
    FROM YSL_data_OK_date
),
YSL_data_theatre_consistent AS (
    SELECT * EXCLUDE(OK), IF(contains(OK, '.'), split_part(OK, '.', 1), OK) AS theatre
    FROM YLS_data_P_replace
),
YSL_data_completeness_OK AS (
    SELECT *
    FROM YSL_data_theatre_consistent
    WHERE theatre IS NOT NULL
),
YSL_data_filter_only_electives AS (
    SELECT *
    FROM YSL_data_completeness_OK
    WHERE P = 'E'
),
YSL_data_turnover AS (
    SELECT *
        , row_number() OVER (PARTITION BY theatre, date ORDER BY actual_end_case_datetime) AS actual_day_sequence_number
        , lag(actual_end_case_datetime) OVER (PARTITION BY theatre, date ORDER BY actual_end_case_datetime) AS actual_previous_end_case_datetime
    FROM YSL_data_filter_only_electives
),
YSL_data_surgeon AS (
    SELECT * EXCLUDE ("Snijder")
        , split("Snijder", ',')[2] AS surgeon_name
        , split("Snijder", ',')[1] AS surgeon_surname
        ,"Verr. code" AS protocol_code
    FROM YSL_data_turnover
),
YSL_data_protocol_join AS (
    SELECT 
        YDS.* EXCLUDE ("Verr. code")
        ,YDS."Spec." AS specialism_name
        ,YDP.procedure_id
        ,YDP.procedure_name
        -- ,YDP.protocol_id
        -- ,YDP.protocol_name
        ,IFNULL(YDP.organization_name, 'IJsselland Ziekenhuis') AS organization_name
        ,IFNULL(YDP.organization_id, 'cc213081-6d8b-4149-89e0-8099d678b243') AS organization_id
    FROM YSL_data_surgeon YDS
    LEFT OUTER JOIN YSL_data_protocol YDP ON YDS.protocol_code = UPPER(YDP.protocol_code)
),
YSL_data_exclude_nonrelevant_columns AS (
    SELECT * EXCLUDE (
        Dag,
        Month,
        P,
        Complex,
        "n. Afd",
        Minutes,
        "Hoofd/neven1",
        Lok,
        "Spec."
    )
    FROM YSL_data_protocol_join
),
YSL_data_format AS (
    SELECT 
        organization_id
        ,organization_name
        ,theatre
        ,actual_day_sequence_number
        ,NULL AS expected_day_sequence_number
        ,surgeon_surname
        ,surgeon_name
        ,specialism_name
        ,procedure_id
        ,procedure_name
        ,try_cast(NULL AS nvarchar) AS protocol_id
        ,protocol_code
        ,try_cast(NULL AS nvarchar) AS protocol_name
        ,date
        ,actual_start_case_datetime
        ,try_cast(NULL AS timestamp) AS expected_start_case_datetime
        ,actual_end_case_datetime
        ,try_cast(NULL AS timestamp) AS expected_end_case_datetime
        ,actual_previous_end_case_datetime
        ,try_cast(NULL AS timestamp) AS expected_previous_end_case_datetime
        ,actual_start_case_datetime - actual_previous_end_case_datetime AS actual_turnovertime 
        ,try_cast(NULL AS interval) AS expected_turnovertime
        ,minute(actual_turnovertime) + 60 * hour(actual_turnovertime) AS actual_turnovertime_minutes
        ,minute(expected_turnovertime) + 60 * hour(expected_turnovertime) AS expected_turnovertime_minutes
    FROM YSL_data_exclude_nonrelevant_columns
)
SELECT *
FROM YSL_data_format


Running query in 'duckdb'

Count
7241


In [8]:
%%sql 
DESCRIBE YSL_parsed_data

Running query in 'duckdb'

column_name,column_type,null,key,default,extra
organization_id,VARCHAR,YES,None,None,None
organization_name,VARCHAR,YES,None,None,None
theatre,VARCHAR,YES,None,None,None
actual_day_sequence_number,BIGINT,YES,None,None,None
expected_day_sequence_number,INTEGER,YES,None,None,None
surgeon_surname,VARCHAR,YES,None,None,None
surgeon_name,VARCHAR,YES,None,None,None
specialism_name,VARCHAR,YES,None,None,None
procedure_id,BIGINT,YES,None,None,None
procedure_name,VARCHAR,YES,None,None,None


In [31]:
%%sql
COPY YSL_parsed_data TO '../YSL_data_parsed.csv'

Running query in 'duckdb'

Count
7241


In [24]:
%%sql
DROP TABLE IF EXISTS YSL_parsed_efficiency_data;
CREATE TABLE YSL_parsed_efficiency_data AS
WITH YSL_data_expected_day AS (
    SELECT 
        *, 
        CAST(date AS TIMESTAMP) + INTERVAL 8 HOUR AS expected_day_start,
        CAST(date AS TIMESTAMP) + INTERVAL 16 HOUR AS expected_day_end 
    FROM YSL_parsed_data
    WHERE actual_start_case_datetime IS NOT NULL
),
YSL_data_difference_day AS (
    SELECT 
        *,
        IF(
            actual_start_case_datetime < expected_day_start, 
            expected_day_start - actual_start_case_datetime, 
            actual_start_case_datetime - expected_day_start
        ) AS difference_day_start,
        IF(
            actual_end_case_datetime < expected_day_end, 
            expected_day_end - actual_end_case_datetime, 
            actual_end_case_datetime - expected_day_end
        ) AS difference_day_end,
        IF(
            actual_start_case_datetime < expected_day_start, 
            'Before', 
            'After'
        ) AS day_start_difference_type,
        IF(
            actual_end_case_datetime < expected_day_end, 
            'Before',
            'After'
        ) AS day_end_difference_type,
        (expected_day_start < actual_start_case_datetime)::INT AS day_start_overtime_indicator,
        (actual_start_case_datetime < expected_day_end AND expected_day_end < actual_end_case_datetime)::INT AS day_end_overtime_indicator
    FROM YSL_data_expected_day
),
YSL_data_casetime AS (
    SELECT *
    , actual_end_case_datetime - actual_start_case_datetime AS case_duration
    , hour(case_duration) * 60 + minute(case_duration) AS case_duration_minutes
    FROM YSL_data_difference_day
),
YSL_data_min_day_start_day_end AS (
    SELECT 
        theatre,
        date,
        first(day_start_difference_type ORDER BY actual_day_sequence_number ASC) AS day_start_timeline_type,
        first(day_end_difference_type ORDER BY day_end_overtime_indicator DESC, difference_day_end ASC) AS day_end_timeline_type,
        first(actual_start_case_datetime ORDER BY actual_day_sequence_number ASC) AS actual_day_start,
        first(actual_end_case_datetime ORDER BY day_end_overtime_indicator DESC, difference_day_end ASC) AS actual_day_end,
        first(expected_day_start) AS expected_day_start,
        first(expected_day_end) AS expected_day_end,
        first(specialism_name ORDER BY actual_day_sequence_number ASC) AS day_start_specialism,
        first(specialism_name ORDER BY day_end_overtime_indicator DESC, difference_day_end ASC) AS day_end_specialism,
        max(day_start_overtime_indicator) AS day_start_overtime_indicator,
        max(day_end_overtime_indicator) AS day_end_overtime_indicator,
        first(difference_day_start ORDER BY actual_day_sequence_number ASC) AS difference_day_start, 
        min(difference_day_end) AS difference_day_end,
        sum(case_duration_minutes) AS total_case_duration_minutes,
        sum(actual_turnovertime_minutes) AS total_turnovertime_minutes,
        count(*) AS number_of_cases
    FROM YSL_data_casetime
    GROUP BY theatre, date
),
YSL_data_difference_minutes AS (
    SELECT 
        *,
        minute(difference_day_start) + hour(difference_day_start) * 60 AS difference_minutes_day_start,
        minute(difference_day_end) + hour(difference_day_end) * 60 AS difference_minutes_day_end
    FROM YSL_data_min_day_start_day_end
),
YSL_data_format AS (
    SELECT 
        theatre,
        date,
        total_case_duration_minutes,
        total_turnovertime_minutes,
        number_of_cases,
        actual_day_start,
        expected_day_start,
        actual_day_end,
        expected_day_end,
        day_start_timeline_type,
        day_end_timeline_type,
        day_start_specialism,
        day_end_specialism,
        day_start_overtime_indicator,
        day_end_overtime_indicator,
        difference_day_start,
        difference_day_end,
        difference_minutes_day_start,
        difference_minutes_day_end
    FROM
        YSL_data_difference_minutes
)
SELECT *
FROM YSL_data_format
ORDER BY theatre DESC, date

Running query in 'duckdb'

Count
1718


In [22]:
%%sql
SELECT *
FROM YSL_parsed_efficiency_data
WHERE day_start_timeline_type = 'Before'
LIMIT 10

Running query in 'duckdb'

theatre,date,total_case_duration_minutes,number_of_cases,actual_day_start,expected_day_start,actual_day_end,expected_day_end,day_start_timeline_type,day_end_timeline_type,day_start_specialism,day_end_specialism,day_start_overtime_indicator,day_end_overtime_indicator,difference_day_start,difference_day_end,difference_minutes_day_start,difference_minutes_day_end
8,2023-01-25,244,4,2023-01-25 07:53:00,2023-01-25 08:00:00,2023-01-25 12:30:00,2023-01-25 16:00:00,Before,Before,KNO,KNO,1,0,0:07:00,3:30:00,7,210
8,2023-03-06,252,3,2023-03-06 07:59:00,2023-03-06 08:00:00,2023-03-06 15:00:00,2023-03-06 16:00:00,Before,Before,OOG,OOG,1,0,0:01:00,1:00:00,1,60
8,2023-04-05,191,9,2023-04-05 07:58:00,2023-04-05 08:00:00,2023-04-05 12:33:00,2023-04-05 16:00:00,Before,Before,KNO,KNO,1,0,0:02:00,3:27:00,2,207
7,2023-01-20,314,4,2023-01-20 07:59:00,2023-01-20 08:00:00,2023-01-20 13:52:00,2023-01-20 16:00:00,Before,Before,PLA,PLA,1,0,0:01:00,2:08:00,1,128
7,2023-01-23,185,8,2023-01-23 07:59:00,2023-01-23 08:00:00,2023-01-23 11:36:00,2023-01-23 16:00:00,Before,Before,KNO,KNO,1,0,0:01:00,4:24:00,1,264
7,2023-01-24,253,5,2023-01-24 07:52:00,2023-01-24 08:00:00,2023-01-24 13:33:00,2023-01-24 16:00:00,Before,Before,PLA,PLA,1,0,0:08:00,2:27:00,8,147
7,2023-02-21,188,4,2023-02-21 07:56:00,2023-02-21 08:00:00,2023-02-21 11:39:00,2023-02-21 16:00:00,Before,Before,PLA,PLA,1,0,0:04:00,4:21:00,4,261
7,2023-04-06,217,3,2023-04-06 07:58:00,2023-04-06 08:00:00,2023-04-06 13:19:00,2023-04-06 16:00:00,Before,Before,CHI,CHI,1,0,0:02:00,2:41:00,2,161
7,2023-04-20,325,11,2023-04-20 07:58:00,2023-04-20 08:00:00,2023-04-20 14:49:00,2023-04-20 16:00:00,Before,Before,KNO,KNO,1,0,0:02:00,1:11:00,2,71
7,2023-07-04,247,3,2023-07-04 07:59:00,2023-07-04 08:00:00,2023-07-04 12:39:00,2023-07-04 16:00:00,Before,Before,PLA,PLA,1,0,0:01:00,3:21:00,1,201


In [25]:
%%sql 
COPY YSL_parsed_efficiency_data TO '../YSL_parsed_efficiency_data_v1.csv'


Running query in 'duckdb'

Count
1718


In [20]:
%%sql
WITH sum_date AS (
    SELECT date, theatre, SUM(actual_turnovertime_minutes) AS total_turnovertime_minutes
    FROM YSL_parsed_data
    GROUP BY date, theatre
)
SELECT year(sd.date), month(sd.date), AVG(total_turnovertime_minutes) / 60
FROM sum_date sd 
INNER JOIN YSL_parsed_efficiency_data df ON df.date = sd.date AND df.theatre = sd.theatre
WHERE 1=1 
    AND df.day_start_timeline_type = 'After' AND difference_minutes_day_start < 120
    AND df.day_end_timeline_type = 'After' AND difference_minutes_day_end < 120
GROUP BY year(sd.date), month(sd.date)

Running query in 'duckdb'

"""year""(sd.date)","""month""(sd.date)",(avg(total_turnovertime_minutes) / 60)
2023,1,1.3194444444444444
2023,2,1.35
2023,3,1.2166666666666666
2023,4,1.2595238095238095
2023,5,1.5958333333333334
2023,6,1.0479166666666666
2023,7,2.9541666666666666
2023,8,1.5611111111111111
2023,9,2.077777777777778
2023,10,1.375925925925926
